# Transaction Management

```{important}
This notebook is in the process of being migrated to Vast Data Platform Field Docs.  It will probably not run yet.
```

```{seealso}
The Vast DB SDK API Documentation is available [here](https://vastdb-sdk.readthedocs.io).
```

VAST-DB is ACID compliant. All operations in VAST-DB are transactions, thus the boundaries of these transactions can be moved at will to cover very complex activities. The nature of how data is managed in the metastore makes transaction and transaction roll-back very low-latency. These mechanics are covered in the “Element Store” section.

## Install sdk and connect to Vast DB

Install vastdb library.

In [ ]:
!pip install --quiet vastdb

In [ ]:
# This cell is used for automated testing.  It will be removed when published.

import os

ENDPOINT = os.environ['ENDPOINT']
ACCESS_KEY = os.environ['ACCESS_KEY']
SECRET_KEY = os.environ['SECRET_KEY']
DATABASE_NAME = os.environ['DATABASE_NAME']

In [ ]:
# Change these variables to reflect your environment, E.g. 
#
# ENDPOINT = 'http://your_vast_endpoint:12345'
# DATABASE_NAME = 'your_db'
# ACCESS_KEY = 'your_access_key'
# SECRET_KEY = 'your_secret_key'

Connect to Vast DB

In [ ]:
import vastdb

session = vastdb.connect(
    endpoint=ENDPOINT,
    access=ACCESS_KEY,
    secret=SECRET_KEY)

## Transactions

In [79]:
DATABASE_SCHEMA = 'this_should_never_get_created'

with session.transaction() as tx:

    bucket = tx.bucket(DATABASE_NAME)
    bucket.create_schema(DATABASE_SCHEMA, fail_if_exists=False)

    raise Exception("Forcing a failure inside the transaction - transaction should rollback")
    

rolling back txid=000030000000003a due to:
Traceback (most recent call last):
  File "/tmp/ipykernel_1103/47924204.py", line 8, in <module>
    raise Exception("Forcing a failure inside the transaction - transaction should rollback")
Exception: Forcing a failure inside the transaction - transaction should rollback


Exception: Forcing a failure inside the transaction - transaction should rollback

In the exception above, you should see `rolling back txid=...` indicating that the transaction was rolled back.

Now let's print out the schemas to verify that it wasn't created:


In [ ]:
with session.transaction() as tx:

    bucket = tx.bucket(DATABASE_NAME)
    schemas = bucket.schemas()
    print(schemas)